In [73]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [74]:
df = pd.read_csv("Dataset/training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1');
df.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12976 entries, 0 to 12975
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   essay_id        12976 non-null  int64  
 1   essay_set       12976 non-null  int64  
 2   essay           12976 non-null  object 
 3   rater1_domain1  12976 non-null  int64  
 4   rater2_domain1  12976 non-null  int64  
 5   rater3_domain1  128 non-null    float64
 6   domain1_score   12976 non-null  int64  
 7   rater1_domain2  1800 non-null   float64
 8   rater2_domain2  1800 non-null   float64
 9   domain2_score   1800 non-null   float64
 10  rater1_trait1   2292 non-null   float64
 11  rater1_trait2   2292 non-null   float64
 12  rater1_trait3   2292 non-null   float64
 13  rater1_trait4   2292 non-null   float64
 14  rater1_trait5   723 non-null    float64
 15  rater1_trait6   723 non-null    float64
 16  rater2_trait1   2292 non-null   float64
 17  rater2_trait2   2292 non-null  

In [76]:
df.shape

(12976, 28)

In [77]:
df.isnull().sum()

essay_id              0
essay_set             0
essay                 0
rater1_domain1        0
rater2_domain1        0
rater3_domain1    12848
domain1_score         0
rater1_domain2    11176
rater2_domain2    11176
domain2_score     11176
rater1_trait1     10684
rater1_trait2     10684
rater1_trait3     10684
rater1_trait4     10684
rater1_trait5     12253
rater1_trait6     12253
rater2_trait1     10684
rater2_trait2     10684
rater2_trait3     10684
rater2_trait4     10684
rater2_trait5     12253
rater2_trait6     12253
rater3_trait1     12848
rater3_trait2     12848
rater3_trait3     12848
rater3_trait4     12848
rater3_trait5     12848
rater3_trait6     12848
dtype: int64

In [78]:
df[df['essay_set']==7]

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
10684,17834,7,Patience is when your waiting .I was patience ...,8,7,NaN,15,NaN,NaN,NaN,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10685,17836,7,"I am not a patience person, like I cant sit i...",6,7,NaN,13,NaN,NaN,NaN,...,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10686,17837,7,One day I was at basketball practice and I was...,7,8,NaN,15,NaN,NaN,NaN,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10687,17838,7,I going to write about a time when I went to t...,8,9,NaN,17,NaN,NaN,NaN,...,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10688,17839,7,It can be very hard for somebody to be patient...,7,6,NaN,13,NaN,NaN,NaN,...,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12248,19558,7,One time I was getting a cool @CAPS1 game it w...,6,6,NaN,12,NaN,NaN,NaN,...,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12249,19559,7,A patent person in my life is my mom. Aicason ...,9,7,NaN,16,NaN,NaN,NaN,...,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12250,19561,7,A time when someone else I know was patient wa...,11,8,NaN,19,NaN,NaN,NaN,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12251,19562,7,I hate weddings. I love when people get marrie...,12,10,NaN,22,NaN,NaN,NaN,...,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
df.dropna(axis=1,inplace=True)
df.shape

(12976, 6)

In [80]:
df = df.iloc[:500]
df.shape

(500, 6)

In [81]:
df.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",4,4,8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,10
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,8


In [82]:
df['essay_set'].value_counts()

essay_set
1    500
Name: count, dtype: int64

In [83]:
df.drop(columns=['rater1_domain1','rater2_domain1'],inplace=True,axis=1)
df.dropna(subset=['essay', 'domain1_score'], inplace=True)

# Exclude the 'essay' column
numeric_df = df.select_dtypes(include='number')

In [84]:
numeric_df.shape

(500, 3)

In [85]:
numeric_df.head()

,essay_id,essay_set,domain1_score
0,1,1,8
1,2,1,9
2,3,1,7
3,4,1,10
4,5,1,8


In [86]:
min(numeric_df['domain1_score'])

2

In [87]:
max(numeric_df['domain1_score'])

12

In [88]:
numeric_df.skew()

essay_id         0.005714
essay_set        0.000000
domain1_score   -0.497040
dtype: float64

In [89]:
numeric_df.kurtosis()

essay_id        -1.203358
essay_set        0.000000
domain1_score    1.426665
dtype: float64

In [90]:
numeric_df[numeric_df['essay_set']==2]['domain1_score']

Series([], Name: domain1_score, dtype: int64)

In [91]:
min_range = [2,1,0,0,0,0,0,0]
max_range = [12,6,3,3,4,4,30,60]

def normalize(x,mi,ma):
    #print("Before Normalization: "+str(x))
    x = (x-mi)/(ma-mi)
    #print("After Normalization : "+str(x))
    return round(x*10)

numeric_df['final_score']=numeric_df.apply(lambda x:normalize(x['domain1_score'],min_range[x['essay_set']-1],max_range[x['essay_set']-1]),axis=1)

In [92]:
numeric_df.drop('domain1_score',axis=1,inplace=True)

In [93]:
# Bring back the 'essay' column
df = pd.concat([df.drop(columns=numeric_df.columns, errors='ignore'), numeric_df], axis=1)
df.head()

,essay,domain1_score,essay_id,essay_set,final_score
0,"Dear local newspaper, I think effects computer...",8,1,1,6
1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,2,1,7
2,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,3,1,5
3,"Dear Local Newspaper, @CAPS1 I have found that...",10,4,1,8
4,"Dear @LOCATION1, I know having computers has a...",8,5,1,6


In [94]:
def clean_essay(essay):
    x=[]
    for i in essay.split():
        if i.startswith("@"):
            continue
        else:
            x.append(i)
    return ' '.join(x)

df['essay'] = df['essay'].apply(lambda x:clean_essay(x))

In [95]:
stop_words = set(stopwords.words('english')) 
def remove_stop_words(essay):
    word_tokens = word_tokenize(essay) 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    return ' '.join(filtered_sentence)

df['clean_essay'] = df['essay'].apply(lambda x:remove_stop_words(x))

In [96]:
def remove_puncs(essay):
    essay = re.sub("[^A-Za-z ]","",essay)
    return essay

df['clean_essay'] = df['clean_essay'].apply(lambda x:remove_puncs(x))

In [97]:
df.head()

,essay,domain1_score,essay_id,essay_set,final_score,clean_essay
0,"Dear local newspaper, I think effects computer...",8,1,1,6,Dear local newspaper I think effects computer...
1,Dear I believe that using computers will benef...,9,2,1,7,Dear I believe using computers benefit us many...
2,"Dear, More and more people use computers, but ...",7,3,1,5,Dear More people use computers everyone agre...
3,"Dear Local Newspaper, I have found that many e...",10,4,1,8,Dear Local Newspaper I found many experts say...
4,Dear I know having computers has a positive ef...,8,5,1,6,Dear I know computers positive effect people ...


In [98]:
def sent2word(x):
    x=re.sub("[^A-Za-z0-9]"," ",x)
    words=nltk.word_tokenize(x)
    return words

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words
        

def noOfWords(essay):
    count=0
    for i in essay2word(essay):
        count=count+len(i)
    return count

def noOfChar(essay):
    count=0
    for i in essay2word(essay):
        for j in i:
            count=count+len(j)
    return count

def avg_word_len(essay):
    return noOfChar(essay)/noOfWords(essay)

def noOfSent(essay):
    return len(essay2word(essay))

def count_pos(essay):
    sentences = essay2word(essay)
    noun_count=0
    adj_count=0
    verb_count=0
    adverb_count=0
    for i in sentences:
        pos_sentence = nltk.pos_tag(i)
        for j in pos_sentence:
            pos_tag = j[1]
            if(pos_tag[0]=='N'):
                noun_count+=1
            elif(pos_tag[0]=='V'):
                verb_count+=1
            elif(pos_tag[0]=='J'):
                adj_count+=1
            elif(pos_tag[0]=='R'):
                adverb_count+=1
    return noun_count,verb_count,adj_count,adverb_count

data = open('big.txt').read()
words = re.findall('[a-z]+', data.lower())

def check_spell_error(essay):
    essay=essay.lower()
    new_essay = re.sub("[^A-Za-z0-9]"," ",essay)
    new_essay = re.sub("[0-9]","",new_essay)
    count=0
    all_words = new_essay.split()
    for i in all_words:
        if i not in words:
            count+=1
    return count
    
    

In [99]:
# pro_data = df.copy()
# pro_data['char_count'] = pro_data['essay'].apply(noOfChar)
# pro_data['word_count'] = pro_data['essay'].apply(noOfWords)
# pro_data['sent_count'] = pro_data['essay'].apply(noOfSent)
# pro_data['avg_word_len'] = pro_data['essay'].apply(avg_word_len)
# pro_data['spell_err_count'] = pro_data['essay'].apply(check_spell_error)
# pro_data['noun_count'], pro_data['adj_count'], pro_data['verb_count'], pro_data['adv_count'] = zip(*pro_data['essay'].map(count_pos))
# pro_data.to_csv("Processed_data.csv")

In [100]:
prep_df = pd.read_csv("Processed_data.csv")
prep_df.drop('Unnamed: 0',inplace=True,axis=1)
prep_df.head()

,essay,domain1_score,essay_id,essay_set,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,"Dear local newspaper, I think effects computer...",8,1,1,6,Dear local newspaper I think effects computer...,1441,344,16,4.188953,11,76,75,18,24
1,Dear I believe that using computers will benef...,9,2,1,7,Dear I believe using computers benefit us many...,1765,413,17,4.273608,21,98,84,20,19
2,"Dear, More and more people use computers, but ...",7,3,1,5,Dear More people use computers everyone agre...,1185,276,14,4.293478,5,76,51,20,16
3,"Dear Local Newspaper, I have found that many e...",10,4,1,8,Dear Local Newspaper I found many experts say...,2284,490,26,4.661224,31,142,96,39,29
4,Dear I know having computers has a positive ef...,8,5,1,6,Dear I know computers positive effect people ...,2023,469,30,4.313433,18,110,90,32,36


In [101]:
from sklearn.preprocessing import LabelEncoder

In [102]:
vectorizer = CountVectorizer(max_features = 10000, ngram_range=(1, 3), stop_words='english')
count_vectors = vectorizer.fit_transform(df['clean_essay'])
feature_names = vectorizer.get_feature_names_out()
data = df[['essay_set','clean_essay','final_score']].copy()
X = count_vectors.toarray()
y = data['final_score'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)



# # Vectorize the text data
# vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 3), stop_words='english')
# count_vectors = vectorizer.fit_transform(prep_df['clean_essay'])
# X_text = count_vectors.toarray()

# # Get additional features (ensure all are numeric)
# additional_features = prep_df.iloc[:, 5:]
# # Check if there are any non-numeric columns and encode them
# for col in additional_features.select_dtypes(include=['object']).columns:
#     le = LabelEncoder()
#     additional_features[col] = le.fit_transform(additional_features[col])

# X_additional = additional_features.values

# # Combine text features with additional features
# X_full = np.concatenate((X_additional, X_text), axis=1)

# # Get the target variable
# y_full = prep_df['final_score'].values

# # Split the dataset into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.3, random_state=42)



SVR

In [103]:
#Save Trained Model
clf = SVR(C=1.0, epsilon=0.2)
clf.fit(X_train, y_train)
pickle.dump(clf,open("Saved_Models/SVR_with_pp",'wb'))

#Use Saved Model
# clf = pickle.load(open('Saved_Models/SVR_with_pp', 'rb'))
# y_pred=clf.predict(X_test)
# print("Mean squared error:%.2f"%mean_squared_error(y_test,y_pred))

In [104]:
y_pred=clf.predict(X_test)
print("Mean squared error:%.2f"%mean_squared_error(y_test,y_pred))

Mean squared error:1.33


In [105]:
df.head()

,essay,domain1_score,essay_id,essay_set,final_score,clean_essay
0,"Dear local newspaper, I think effects computer...",8,1,1,6,Dear local newspaper I think effects computer...
1,Dear I believe that using computers will benef...,9,2,1,7,Dear I believe using computers benefit us many...
2,"Dear, More and more people use computers, but ...",7,3,1,5,Dear More people use computers everyone agre...
3,"Dear Local Newspaper, I have found that many e...",10,4,1,8,Dear Local Newspaper I found many experts say...
4,Dear I know having computers has a positive ef...,8,5,1,6,Dear I know computers positive effect people ...


In [117]:
df = pd.read_csv("Dataset/dummy_data.csv", sep='\t', encoding='ISO-8859-1');
df.head()

,"essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,rater1_trait5,rater1_trait6,rater2_trait1,rater2_trait2,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6"
0,"1,1,""Dear local newspaper, I think effects com..."


In [116]:
df.shape

(1, 1)

In [ ]:
clf = pickle.load(open('Saved_Models/SVR_with_pp', 'rb'))
clf.predict(count_vectors)

ValueError: X has 357 features, but SVR is expecting 10010 features as input.